# Python破解图片验证码


## 1. 导入库 

In [ ]:
# 导入PIL
from PIL import Image
import math
import os

## 2. 打开验证码并展示 

In [26]:
im = Image.open("./pics/验证码.gif")
# 将图片转换为8位像素模式
im.convert("P")

颜色直方图的每一位数字都代表了在图片中含有对应位的颜色的像素的数量。

In [9]:
# 打印颜色直方图
print(im.histogram())

[1, 1, 1, 1, 3, 15, 186, 8, 116, 1, 1, 1, 1, 3, 6, 21, 1, 1, 1, 2, 1, 3, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 132, 18, 10, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 365, 10, 115, 135, 625, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 3. 得到出现次数最多的颜色，(颜色编号, 频数)：

使用sorted对频数排序排序。
sort 是应用在 list 上的方法，sorted 可以对所有可迭代的对象进行排序操作。
list 的 sort 方法返回的是对已经存在的列表进行操作，无返回值，而内建函数 sorted 方法返回的是一个新的 list，而不是在原来的基础上进行的操作。
```
sorted(iterable, cmp=None, key=None, reverse=False)
```
参数说明：
* iterable -- 可迭代对象。
* cmp -- 比较的函数，这个具有两个参数，参数的值都是从可迭代对象中取出，此函数必须遵守的规则为，大于则返回1，小于则返回-1，等于则返回0。
* key -- 主要是用来进行比较的元素，只有一个参数，具体的函数的参数就是取自于可迭代对象中，指定可迭代对象中的一个元素来进行排序。
* reverse -- 排序规则，reverse = True 降序 ， reverse = False 升序（默认）。

In [17]:
his = im.histogram()
values = dict(enumerate(his))
print(sorted(values.items(),key=lambda x:x[1], reverse=True)[:10])

[(73, 625), (69, 365), (6, 186), (72, 135), (56, 132), (8, 116), (71, 115), (15, 21), (57, 18), (5, 15)]


## 4. 将数字颜色置0，其余颜色置255，得到图片im2

In [24]:
im2 = Image.new("P", im.size, 255)
for x in range(im.size[1]):
    for y in range(im.size[0]):
        pix = im.getpixel((y,x))
        if pix == 6 or pix == 8:  # 这两种颜色是数字
            im2.putpixel((y,x),0)
im2  # 显示图片

## 5. 向量空间图像识别
关于向量空间搜索引擎的原理可以参考这篇文章：http://ondoc.logand.com/d/2697/pdf

Don't panic。向量空间搜索引擎名字听上去很高大上其实原理很简单。拿文章里的例子来说：

你有 3 篇文档，我们要怎么计算它们之间的相似度呢？2 篇文档所使用的相同的单词越多，那这两篇文章就越相似！但是这单词太多怎么办，就由我们来选择几个关键单词，选择的单词又被称作特征，每一个特征就好比空间中的一个维度（x，y，z 等），一组特征就是一个矢量，每一个文档我们都能得到这么一个矢量，只要计算矢量之间的夹角就能得到文章的相似度了。

下面的类会比较两个 python 字典类型并输出它们的相似度（用 0～1 的数字表示）

In [25]:
# 用python类实现向量相关性系数计算
class VectorCompare:
    # 计算矢量的大小
    def magnitude(self, vector):
        total = 0
        for val in vector.itervalues():
            total += val**2
        return math.sqrt(total)
    
    # 计算矢量之间的cos
    def relation(self, vec1, vec2):
        top = 0
        for key, val in vec1.iteritems():
            if vec2.has_key(key):
                top += val*vec2[key]
        return top / (self.magnitude(vec1) * self.magnitude(vec2))
    
# 实例化相关系数计算类
v = VectorCompare()

## 6. 图片转矢量方法

In [ ]:
# 将图片转换为矢量，矢量用字典保存，key为像素位置，val为像素值
def buildvector(im):
    d1 = {}
    count = 0
    for i in im.getdata():
        d1[count] = i
        count += 1
    return d1

## 7. 加载对比集
加载对比集到imageset列表中，列表的每一项是一个字典，如 {'a': [img转换成的vector]}

In [ ]:
# 加载对比集
# icon set字符集
iconset = ['0','1','2','3','4','5','6','7','8','9','0','a','b',
           'c','d','e','f','g','h','i','j','k','l','m','n','o',
           'p','q','r','s','t','u','v','w','x','y','z']
imageset = []
for letter in iconset:
    for img in os.listdir('./iconset/%s/'%(letter)):
        temp = []
        if img != "Thumbs.db" and img != ".DS_Store":
            temp.append(buildvector(Image.open("./iconset/%s/%s"%(letter,img))))
        imageset.append({letter:temp})

## 8. 对验证码进行切割，对每个单字用比较集进行比较，输出最可能的结果 

In [ ]:
# 对验证码图片进行切割,切割位置存在letters中
inletter = False
foundletter=False
start = 0
end = 0
letters = []
for y in range(im2.size[0]): # slice across
    for x in range(im2.size[1]): # slice down
        pix = im2.getpixel((y,x))
        if pix != 255:
            inletter = True

    if foundletter == False and inletter == True:
        foundletter = True
        start = y

    if foundletter == True and inletter == False:
        foundletter = False
        end = y
        letters.append((start,end))
    inletter=False
    
# 对验证码单字进行比较，并输出最可能的一项
for letter in letters:
    # 切割出单字im3
    im3 = im2.crop(( letter[0] , 0, letter[1],im2.size[1] ))

    guess = []

    # 将切割得到的验证码小片段与每个对比集片段进行比较
    for image in imageset:
        for x,y in image.iteritems():
            if len(y) != 0:
                guess.append( ( v.relation(y[0],buildvector(im3)),x) )

    guess.sort(reverse=True)
    print(guess[0])